## DFS
深度搜索的程序模版：

```python
def func():
    res=list()
    
    def dfs(idx, path):
        '''
        dfs函数，因为是闭包，可以直接访问外部res
        :param idx: 对于全局来说，dfs所在的位置
        :param path: 目前为止dfs已探索的路径
        :return:
        '''
        pass
        dfs(idx + 1, path + something)    # 若path是地址引用，需要改成path[:]
        pass

    init_path = None  # 初始探索路径，一般为0，空字串或空列表
    dfs(0, init_path)
    return res
```

个人习惯把```dfs(idx, path)```写在外部函数中构成闭包，这样做的好处是可以减少```dfs```函数的参数量。需要注意的有两点：
- 如果要在```dfs```函数中修改```res```的话，```res```必须是一个地址引用，如```list```；
- 若```path```是一个地址引用，在递归调用```dfs```时需要传入一个副本```path[:]```。

[Number of Islands](https://leetcode.com/problems/number-of-islands/)。求数字矩阵中数字'1'构成的孤岛数量。

思路：

In [ ]:
def numIslands(grid) -> int:
    if not grid:
        return 0

    rows, cols = len(grid), len(grid[0])
    visited = [[False]*cols for _ in range(rows)]

    def bfs(row, col):
        if row < 0 or row >= rows or col < 0 or col >= cols or grid[row][col] == '0' or visited[row][col]:
            return

        visited[row][col] = True

        bfs(row-1, col)
        bfs(row+1, col)
        bfs(row, col-1)
        bfs(row, col+1)

    res = 0
    for row in range(rows):
        for col in range(cols):
            if grid[row][col] == '0' or visited[row][col]:
                continue

            bfs(row, col)
            res += 1

    return res

[Partition to K Equal Sum Subsets](https://leetcode.com/problems/partition-to-k-equal-sum-subsets/)。给一数组与一个正整数$k$，判断该数组能否被划分成和相等的$k$组数。[Matchsticks to Square](https://leetcode.com/problems/matchsticks-to-square/)。给若干火柴棍，以数组表示，每个元素表示火柴棍的长度，判断这些火柴棍能否拼成一个正方形。

思路：虽然该题并不需要求出一个配置，但是还是通过DFS来做。首先易得每一组的和为$sum/k$，若数组中存在比该值大或者除不尽的情况则直接返回False。将数组排序，并初始化$k$个空数组，由大往小开始填充。第二题是第一题$k=4$时的特殊情况。

In [8]:
def canPartitionKSubsets(nums, k: int) -> bool:
    target, rem = divmod(sum(nums), k)
    if rem:
        return False

    nums.sort()
    if nums[-1] > target:
        return False
    while nums and nums[-1] == target:
        nums.pop()
        k -= 1

    init_group = [0]*k

    def dfs(group):
        if not nums:
            return True

        cur_num = nums.pop()
        for i in range(k):
            if group[i]+cur_num <= target:
                group[i] += cur_num
                if dfs(group):
                    return True
                group[i] -= cur_num
        nums.append(cur_num)
        return False

    return dfs(init_group)

[Unique Paths III](https://leetcode.com/problems/unique-paths-iii/)。给以迷宫，其中$1$表示起点，$2$表示终点，$0$表示可通行，$-1$表示障碍物。求从起点到终点并访问一次所有$0$的位置，有多少种走法。

思路：常规的DFS题。首先统计需要访问的点(0)的数目，然后找到起点开始dfs。

In [1]:
def uniquePathsIII(grid):
    rows, cols = len(grid), len(grid[0])
    cnt = 0
    for row in range(rows):
        for col in range(cols):
            if grid[row][col] == 0:
                cnt += 1

    res = 0
    visited = [[False]*cols for _ in range(rows)]

    def dfs(row, col, path):
        nonlocal res

        if row < 0 or row >= rows or col < 0 or col >= cols \
                or grid[row][col] == -1 or visited[row][col]:
            return
        if grid[row][col] == 2:
            if path == cnt+1:
                res += 1
            return

        directions = [(0, 1), (0, -1), (-1, 0), (1, 0)]
        visited[row][col] = True
        for x, y in directions:
            dfs(row+x, col+y, path+1)
        visited[row][col] = False

    for row in range(rows):
        for col in range(cols):
            if grid[row][col] == 1:
                dfs(row, col, 0)
                return res